## 4.1 Removal of duplicate reactions

The draft EcN model has a too high growth on glucose. Find duplicate reactions to remove all unwanted redundancies. Within this notebook the model is checked for
- Copies of the same reaction
- Different versions of reactions
- Energy generating cycles
- Other causes of inflated growth
- Remaining duplicates

In [1]:
#import package needed
import cobra
import pandas as pd
from cobra.io import load_json_model
from glob import glob
from cobra.manipulation.delete import delete_model_genes

In [2]:
#Load EcN model 
EcN_ID = 'CP022686.1'
model = load_json_model('../data/models/%s.json'%EcN_ID)
modelCopy = model.copy()

In [3]:
# Load the table with information on origin of reactions
rxn_origin_df = pd.read_csv('../tables/rxn_origin.csv')
rxn_origin_df.set_index('reaction', inplace=True)
rxn_origin_df.head()

,origin,added,notebook,notes
reaction,,,,
ALATA_D2,iML1515,automatic,3.1,none
SHCHD2,iML1515,automatic,3.1,none
CPPPGO,iML1515,automatic,3.1,none
GTHOr,iML1515,automatic,3.1,none
DHORD5,iML1515,automatic,3.1,none


## 1. Reaction overview

In [4]:
# Create a dataframe with the ID, formula, compartment and associated genes of all reactions
reaction_df = pd.DataFrame(columns=['ID','formula', 'compartment', 'genes'])

for reaction in modelCopy.reactions:
    formula = reaction.reaction
    ID = reaction.id
    compartment = reaction.compartments
    genes = reaction.genes
    data = {'ID': ID, 'formula': formula, 'compartment': compartment, 'genes': genes}
    reaction_df = reaction_df.append(data, ignore_index=True)

reaction_df.head()

,ID,formula,compartment,genes
0,ALATA_D2,ala__D_c + pydx5p_c --> pyam5p_c + pyr_c,{c},"(CIW80_22360, CIW80_06560)"
1,SHCHD2,dscl_c + nad_c --> h_c + nadh_c + scl_c,{c},(CIW80_11380)
2,CPPPGO,cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 ...,{c},(CIW80_06055)
3,GTHOr,gthox_c + h_c + nadph_c <=> 2.0 gthrd_c + nadp_c,{c},(CIW80_12105)
4,DHORD5,dhor__S_c + mqn8_c --> mql8_c + orot_c,{c},(CIW80_22710)


## 2. Identify duplicate reactions
As a result of the construction method, some reactions were added multiple times. These will be indentified based on their similarity in formula and reduced to one.

In [5]:
# Find all formulas which are present more than once in the model
duplicates = reaction_df[reaction_df['formula'].duplicated()].sort_values('ID')
duplicates

,ID,formula,compartment,genes
2862,12PPDStpp_copy2,12ppd__S_p <=> 12ppd__S_c,"{c, p}",(CIW80_14645)
2807,ADNt2pp,adn_p + h_p --> adn_c + h_c,"{c, p}","(CIW80_08585, CIW80_05845)"
2760,ALAt2pp_copy1,ala__L_p + h_p --> ala__L_c + h_c,"{c, p}",(CIW80_16415)
2756,ASPt2pp_copy1,asp__L_p + h_p --> asp__L_c + h_c,"{c, p}",(CIW80_15645)
2744,CTECOAI6_copy1,tdecoa_c <=> td2coa_c,{c},(CIW80_14210)
2746,CTECOAI7_copy1,hdcoa_c <=> hdd2coa_c,{c},(CIW80_14210)
2742,CTECOAI8_copy1,odecoa_c <=> od2coa_c,{c},(CIW80_14210)
2772,DADK_copy1,atp_c + damp_c <=> adp_c + dadp_c,{c},(CIW80_20510)
2771,DADK_copy2,atp_c + damp_c <=> adp_c + dadp_c,{c},(CIW80_20510)
2808,FADRx_copy1,fad_c + h_c + nadh_c --> fadh2_c + nad_c,{c},(CIW80_14200)


In [6]:
# There might be more reactions that are similar, but have different reversibility
# Therefore, instead find all the genes that are labelled "copy"
gene_copies = reaction_df[reaction_df['ID'].str.contains("copy")].sort_values('ID')
gene_copies

,ID,formula,compartment,genes
2862,12PPDStpp_copy2,12ppd__S_p <=> 12ppd__S_c,"{c, p}",(CIW80_14645)
2833,2HH24DDH1_copy1,2hh24dd_c + h2o_c --> 24dhhed_c,{c},(CIW80_04280)
2834,2HH24DDH1_copy2,2hh24dd_c + h2o_c <=> 24dhhed_c,{c},(CIW80_04280)
825,ADNt2pp_copy1,adn_p + h_p --> adn_c + h_c,"{c, p}","(CIW80_08585, CIW80_05845)"
826,ADNt2pp_copy2,adn_p + h_p <=> adn_c + h_c,"{c, p}",(CIW80_05910)
2760,ALAt2pp_copy1,ala__L_p + h_p --> ala__L_c + h_c,"{c, p}",(CIW80_16415)
2756,ASPt2pp_copy1,asp__L_p + h_p --> asp__L_c + h_c,"{c, p}",(CIW80_15645)
2744,CTECOAI6_copy1,tdecoa_c <=> td2coa_c,{c},(CIW80_14210)
2743,CTECOAI6_copy2,tdecoa_c --> td2coa_c,{c},(CIW80_14210)
2746,CTECOAI7_copy1,hdcoa_c <=> hdd2coa_c,{c},(CIW80_14210)


## 3. Remove duplicates (copies)

### 12PPDStpp

There are two versions:
- Original: Reversible reaction, but without GPR
- Copy2: Reversible reaction, but with gene

In [7]:
modelCopy.reactions.get_by_id('12PPDStpp')

Reaction identifier,12PPDStpp
Name,"(S)-Propane-1,2-diol facilitated transport (periplasm)"
Memory address,0x01ed41bc5d48
Stoichiometry,"12ppd__S_p <=> 12ppd__S_c (S)-Propane-1,2-diol <=> (S)-Propane-1,2-diol"
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [8]:
modelCopy.reactions.get_by_id('12PPDStpp_copy2')

Reaction identifier,12PPDStpp_copy2
Name,"(S)-Propane-1,2-diol facilitated transport (periplasm)"
Memory address,0x01ed41b0c7c8
Stoichiometry,"12ppd__S_p <=> 12ppd__S_c (S)-Propane-1,2-diol <=> (S)-Propane-1,2-diol"
GPR,CIW80_14645
Lower bound,-1000.0
Upper bound,1000.0


The reactions are exactly the same, except that the original is lacking a GPR. Remove the original version and rename copy2

In [9]:
# Remove the original reaction lacking the GPR
modelCopy.remove_reactions(['12PPDStpp'])

# Rename copy2
modelCopy.reactions.get_by_id('12PPDStpp_copy2').id = '12PPDStpp'

In [10]:
# Edit in reaction overview
rxn_origin_df.loc['12PPDStpp', 'added'] = 'removed'
rxn_origin_df.loc['12PPDStpp', 'notes'] = 'GPR was missing. Removed in 4.1. "_copy2" was kept as correct reaction and renamed "12PPDStpp".'

rxn_origin_df.loc['12PPDStpp_copy2', 'notes'] = 'Renamed to 12PPDStpp in 4.1. Original reaction missed GPR and was removed'

In [11]:
modelCopy.reactions.get_by_id('12PPDStpp')

Reaction identifier,12PPDStpp
Name,"(S)-Propane-1,2-diol facilitated transport (periplasm)"
Memory address,0x01ed41b0c7c8
Stoichiometry,"12ppd__S_p <=> 12ppd__S_c (S)-Propane-1,2-diol <=> (S)-Propane-1,2-diol"
GPR,CIW80_14645
Lower bound,-1000.0
Upper bound,1000.0


### 2HH24DDH1

There are two versions:
- Copy1: One-direction reaction, same gene as original
- Copy2: Reversible reaction, same gene as original

The reaction 2HH24DDH has a different reaction product

In [12]:
modelCopy.reactions.get_by_id('2HH24DDH')

Reaction identifier,2HH24DDH
Name,2hh24dd hydratase
Memory address,0x01ed41b1a488
Stoichiometry,2hh24dd_c + h2o_c --> 4h2kpi_c 2-Hydroxyhepta-2-4-dienedioate + H2O H2O --> 4-Hydroxy-2-ketopimelate
GPR,CIW80_04280
Lower bound,0.0
Upper bound,1000.0


In [13]:
modelCopy.reactions.get_by_id('2HH24DDH1_copy1')

Reaction identifier,2HH24DDH1_copy1
Name,2hh24dd hydratase
Memory address,0x01ed41b1a308
Stoichiometry,"2hh24dd_c + h2o_c --> 24dhhed_c 2-Hydroxyhepta-2-4-dienedioate + H2O H2O --> 2,4-Dihydroxyhept-2-enedioate"
GPR,CIW80_04280
Lower bound,0.0
Upper bound,1000.0


In [14]:
modelCopy.reactions.get_by_id('2HH24DDH1_copy2')

Reaction identifier,2HH24DDH1_copy2
Name,2hh24dd hydratase
Memory address,0x01ed41b1a408
Stoichiometry,"2hh24dd_c + h2o_c <=> 24dhhed_c 2-Hydroxyhepta-2-4-dienedioate + H2O H2O <=> 2,4-Dihydroxyhept-2-enedioate"
GPR,CIW80_04280
Lower bound,-1000.0
Upper bound,1000.0


All models on the BiGG database have the reaction in one direction. Remove the second copy

In [15]:
# Remove the second copy
modelCopy.remove_reactions(['2HH24DDH1_copy2'])

# Rename copy1
modelCopy.reactions.get_by_id('2HH24DDH1_copy1').id = '2HH24DDH1'

In [16]:
# Edit in reaction overview
rxn_origin_df.loc['2HH24DDH1_copy2', 'added'] = 'removed'
rxn_origin_df.loc['2HH24DDH1_copy2', 'notes'] = 'Removed in 4.1. "_copy1" was kept as correct reaction and renamed "2HH24DDH1".'

rxn_origin_df.loc['2HH24DDH1_copy1', 'notes'] = 'Renamed to 2HH24DDH1 in 4.1.'

### ADNt2pp, CYTDt2pp, INSt2pp, THMDt2pp & URIt2pp

For ADNt2pp there are three versions:
- Original: One-direction reaction (fwd)
- Copy1: One-direction reaction (fwd), same genes as original
- Copy2: Reversible reaction (but different gene)

All three genes have the product "nucleoside permease" in the genbank file > https://www.ncbi.nlm.nih.gov/nuccore/CP007799.1/

For CYTDt2pp there are two versions:
- Copy1: One-direction reaction
- Copy2: Reversible reaction

For INSt2pp there are three versions:
- Original: One-direction reaction (fwd)
- Copy1: One-direction reaction (fwd), same gene as original, but also missing two that the original has
- Copy2: Bi-directional reaction, same gene as original (other than copy1), but also missing two that the original has

For THMDt2pp there are three versions:
- Original:  One-direction reaction (fwd), with two genes
- Copy1: One-direction reaction (fwd), with same genes as original
- Copy2: Bi-directional reaction, with different genes as original and copy1

For URIt2pp there are two versions:
- Copy1: One-direction reaction
- Copy2: Reversible reaction

All four symport systems are performed by the same genes ('CIW80_08585 or CIW80_05910 or CIW80_05845')

In [17]:
modelCopy.reactions.INSt2pp

Reaction identifier,INSt2pp
Name,Inosine transport in via proton symport (periplasm)
Memory address,0x01ed41c33c08
Stoichiometry,h_p + ins_p --> h_c + ins_c H+ + Inosine --> H+ + Inosine
GPR,CIW80_08585 or CIW80_05910 or CIW80_05845
Lower bound,0.0
Upper bound,1000.0


In [18]:
modelCopy.reactions.INSt2pp_copy1

Reaction identifier,INSt2pp_copy1
Name,Inosine transport in via proton symport (periplasm)
Memory address,0x01ed41b2c508
Stoichiometry,h_p + ins_p --> h_c + ins_c H+ + Inosine --> H+ + Inosine
GPR,CIW80_08585
Lower bound,0.0
Upper bound,1000.0


In [19]:
modelCopy.reactions.INSt2pp_copy2

Reaction identifier,INSt2pp_copy2
Name,Inosine transport in via proton symport (periplasm)
Memory address,0x01ed41b2c6c8
Stoichiometry,h_p + ins_p <=> h_c + ins_c H+ + Inosine <=> H+ + Inosine
GPR,CIW80_05910
Lower bound,-1000.0
Upper bound,1000.0


ADNt2pp: Bi-directional, since it's a nucleoside (https://biocyc.org/compound?orgid=GCF_000714595&id=ADENOSINE#tab=RXNS)<br>
CYTDt2pp: Bi-directional, since it's a nucleoside (https://biocyc.org/compound?orgid=GCF_000714595&id=CYTIDINE#tab=RXNS)<br>
INSt2pp: Bi-directional, since it's a nucleoside (https://biocyc.org/compound?orgid=GCF_000714595&id=INOSINE#tab=RXNS)<br>
THMDt2pp: Bi-directional, since it's a nucleoside (https://biocyc.org/compound?orgid=GCF_000714595&id=THYMIDINE#tab=RXNS)<br>
URIt2pp: Bi-directional, since it's a nucleoside (https://biocyc.org/compound?orgid=GCF_000714595&id=URIDINE#tab=RXNS)<br>

In [20]:
# Remove the one-directional reactions
modelCopy.remove_reactions(['ADNt2pp', 'ADNt2pp_copy1',
                            'CYTDt2pp_copy1', 
                            'INSt2pp', 'INSt2pp_copy1', 
                            'THMDt2pp','THMDt2pp_copy1',
                            'URIt2pp_copy1'])

# Rename the bi-directional reactions
modelCopy.reactions.ADNt2pp_copy2.id = 'ADNt2pp'
modelCopy.reactions.CYTDt2pp_copy2.id = 'CYTDt2pp'
modelCopy.reactions.INSt2pp_copy2.id = 'INSt2pp'
modelCopy.reactions.THMDt2pp_copy2.id = 'THMDt2pp'
modelCopy.reactions.URIt2pp_copy2.id = 'URIt2pp'

# Update the GPR
modelCopy.reactions.ADNt2pp.gene_reaction_rule = 'CIW80_08585 or CIW80_05910 or CIW80_05845'
modelCopy.reactions.CYTDt2pp.gene_reaction_rule = 'CIW80_08585 or CIW80_05910 or CIW80_05845'
modelCopy.reactions.INSt2pp.gene_reaction_rule = 'CIW80_08585 or CIW80_05910 or CIW80_05845'
modelCopy.reactions.THMDt2pp.gene_reaction_rule = 'CIW80_08585 or CIW80_05910 or CIW80_05845'
modelCopy.reactions.URIt2pp.gene_reaction_rule = 'CIW80_08585 or CIW80_05910 or CIW80_05845'

In [21]:
# Update the reaction overview

# "delete" the incorrect genes
for rxn in ['ADNt2pp', 'ADNt2pp_copy1','CYTDt2pp_copy1','INSt2pp', 'INSt2pp_copy1',
            'THMDt2pp','THMDt2pp_copy1', 'URIt2pp_copy1']:
    rxn_origin_df.loc[rxn, 'added'] = 'removed'
    rxn_origin_df.loc[rxn, 'notes'] = 'Duplicate reaction. Removed in 4.1."%s_copy2" was kept as correct reaction.'%rxn
    
# Update the note of "_copy2"
for rxn in ['ADNt2pp_copy2', 'CYTDt2pp_copy2', 'INSt2pp_copy2', 'THMDt2pp_copy2', 'URIt2pp_copy2']:
    rxn_origin_df.loc[rxn, 'notes'] = 'Renamed to %s in 4.1. Original reaction and/or "_copy1" was removed'%rxn

### ALAt2pp, ASPt2pp, and GLYt2pp
ALAt2pp, ASPt2pp, and GLYt2pp each have one copy. In all three cases the original and its copy are completely identical. Therefore the copies are removed

In [22]:
# Remove copies
modelCopy.remove_reactions(['ALAt2pp_copy1', 'ASPt2pp_copy1', 'GLYt2pp_copy1'])

# Edit in reaction overview
for rxn in ['ALAt2pp_copy1', 'ASPt2pp_copy1', 'GLYt2pp_copy1']:
    rxn_origin_df.loc[rxn, 'added'] = 'removed'
    rxn_origin_df.loc[rxn, 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept'

### CTECOAI6, CTECOAI7, CTECOAI8
All three reactions (CTECOAI6, CTECOAI7, CTECOAI8) are linked to the same gene (CIW80_14210 - fabB). In all cases there are three versions:
- Original: Reversible reaction
- Copy1: Reversible reaction, same as original
- Copy2: One-direction reaction (fwd), same gene as copy1

In [23]:
modelCopy.reactions.CTECOAI7

Reaction identifier,CTECOAI7
Name,3-cis-2-trans-enoyl-CoA isomerase
Memory address,0x01ed41b68408
Stoichiometry,hdcoa_c <=> hdd2coa_c Hexadecenoyl-CoA (n-C16:1CoA) <=> Trans-Hexadec-2-enoyl-CoA
GPR,CIW80_14210
Lower bound,-1000.0
Upper bound,1000.0


In [24]:
modelCopy.reactions.CTECOAI8_copy1

Reaction identifier,CTECOAI8_copy1
Name,3-cis-2-trans-enoyl-CoA isomerase
Memory address,0x01ed41b23dc8
Stoichiometry,odecoa_c <=> od2coa_c Octadecenoyl-CoA (n-C18:1CoA) <=> Trans-Octadec-2-enoyl-CoA
GPR,CIW80_14210
Lower bound,-1000.0
Upper bound,1000.0


In [25]:
modelCopy.reactions.CTECOAI8_copy2

Reaction identifier,CTECOAI8_copy2
Name,3-cis-2-trans-enoyl-CoA isomerase
Memory address,0x01ed41b23848
Stoichiometry,odecoa_c --> od2coa_c Octadecenoyl-CoA (n-C18:1CoA) --> Trans-Octadec-2-enoyl-CoA
GPR,CIW80_14210
Lower bound,0.0
Upper bound,1000.0


The gene "CIW80_14210" is present in the Nissle genome (https://www.ncbi.nlm.nih.gov/nuccore/CP007799.1/) and annotated as "fadB". FadB is a multifunctional enzyme that is involved in the degradation of fatty acids via the β-oxidation cycle. Four enzymatic activities are associated with FadB: enoyl-CoA hydratase, 3-hydroxyacyl-CoA epimerase, 3-hydroxyacyl-CoA dehydrogenase, and Δ3-cis- Δ2-trans-enoyl-CoA isomerase. https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS14105

The reactions that do not include cofactors are revisible on BioCyc. Only keep the original reaction.

In [26]:
# Remove the copies
modelCopy.remove_reactions(['CTECOAI6_copy1', 'CTECOAI6_copy2', 
                            'CTECOAI7_copy1', 'CTECOAI7_copy2', 
                            'CTECOAI8_copy1', 'CTECOAI8_copy2'])

In [27]:
# Update in reaction overview
for rxn in ['CTECOAI6_copy1', 'CTECOAI6_copy2', 
            'CTECOAI7_copy1', 'CTECOAI7_copy2', 
            'CTECOAI8_copy1', 'CTECOAI8_copy2']:
    rxn_origin_df.loc[rxn, 'added'] = 'removed'
    rxn_origin_df.loc[rxn, 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept'

### DADK

All three version are exactly the same

In [28]:
modelCopy.reactions.DADK

Reaction identifier,DADK
Name,Deoxyadenylate kinase
Memory address,0x01ed41c07a88
Stoichiometry,atp_c + damp_c <=> adp_c + dadp_c ATP C10H12N5O13P3 + DAMP C10H12N5O6P <=> ADP C10H12N5O10P2 + DADP C10H12N5O9P2
GPR,CIW80_20510
Lower bound,-1000.0
Upper bound,1000.0


In [29]:
modelCopy.reactions.DADK_copy1

Reaction identifier,DADK_copy1
Name,Deoxyadenylate kinase
Memory address,0x01ed41b23748
Stoichiometry,atp_c + damp_c <=> adp_c + dadp_c ATP C10H12N5O13P3 + DAMP C10H12N5O6P <=> ADP C10H12N5O10P2 + DADP C10H12N5O9P2
GPR,CIW80_20510
Lower bound,-1000.0
Upper bound,1000.0


In [30]:
modelCopy.reactions.DADK_copy2

Reaction identifier,DADK_copy2
Name,Deoxyadenylate kinase
Memory address,0x01ed41b23308
Stoichiometry,atp_c + damp_c <=> adp_c + dadp_c ATP C10H12N5O13P3 + DAMP C10H12N5O6P <=> ADP C10H12N5O10P2 + DADP C10H12N5O9P2
GPR,CIW80_20510
Lower bound,-1000.0
Upper bound,1000.0


In [31]:
# Remove the two copies
modelCopy.remove_reactions(['DADK_copy1', 'DADK_copy2'])

In [32]:
# Update in reaction overview
for rxn in ['DADK_copy1', 'DADK_copy2']:
    rxn_origin_df.loc[rxn, 'added'] = 'removed'
    rxn_origin_df.loc[rxn, 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept'

### FADRx

There are three versions:
- Original: One-direction reaction (fwd)
- Copy1: One-direction reaction (fwd), same genes as original
- Copy2: One-direction reaction (fwd), but different gene

In [33]:
modelCopy.reactions.FADRx

Reaction identifier,FADRx
Name,FAD reductase
Memory address,0x01ed41cb6648
Stoichiometry,fad_c + h_c + nadh_c --> fadh2_c + nad_c Flavin adenine dinucleotide oxidized + H+ + Nicotinamide adenine dinucleotide - reduced --> Flavin adenine dinucleotide reduced + Nicotinamide adenine dinucleotide
GPR,CIW80_14200
Lower bound,0.0
Upper bound,1000.0


In [34]:
modelCopy.reactions.FADRx_copy1

Reaction identifier,FADRx_copy1
Name,FAD reductase
Memory address,0x01ed41b1aa48
Stoichiometry,fad_c + h_c + nadh_c --> fadh2_c + nad_c Flavin adenine dinucleotide oxidized + H+ + Nicotinamide adenine dinucleotide - reduced --> Flavin adenine dinucleotide reduced + Nicotinamide adenine dinucleotide
GPR,CIW80_14200
Lower bound,0.0
Upper bound,1000.0


In [35]:
modelCopy.reactions.FADRx_copy2

Reaction identifier,FADRx_copy2
Name,FAD reductase
Memory address,0x01ed41b1a1c8
Stoichiometry,fad_c + h_c + nadh_c --> fadh2_c + nad_c Flavin adenine dinucleotide oxidized + H+ + Nicotinamide adenine dinucleotide - reduced --> Flavin adenine dinucleotide reduced + Nicotinamide adenine dinucleotide
GPR,CIW80_04305
Lower bound,0.0
Upper bound,1000.0


In [36]:
modelCopy.genes.CIW80_14200

Gene identifier,CIW80_14200
Name,fre
Memory address,0x01ed41dbf808
Functional,True
In 7 reaction(s),"FMNRx2, FADRx, FADRx_copy1, FLVRx, FLVR, FE3Ri, FMNRx"


In BioCyc CIW80_14200 is annotated as Flavin reductase (fre). https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS14095
<br>CIW80_04305 is annotated as hydroxyethylthiazole kinase (thiM), a hydroxyethylthiazole kinase that catalyses a completely different reaction.https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS04300

Only keep the original reaction.

In [37]:
# Remove the duplicates
modelCopy.remove_reactions(['FADRx_copy1', 'FADRx_copy2'])

In [38]:
# Update in reaction overview
rxn_origin_df.loc['FADRx_copy1', 'added'] = 'removed'
rxn_origin_df.loc['FADRx_copy1', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

rxn_origin_df.loc['FADRx_copy2', 'added'] = 'removed'
rxn_origin_df.loc['FADRx_copy2', 'notes'] = 'Incorrect gene. Removed in 4.1. Original was kept.'

### FEENTERtex

There are three versions:
- Original: One-direction reaction (fwd)
- Copy1: One-direction reaction (fwd), same genes as original
- Copy2: One-direction reaction (fwd), but different gene

In [39]:
modelCopy.reactions.FEENTERtex

Reaction identifier,FEENTERtex
Name,Enterochelin transport (secretion periplasm)
Memory address,0x01ed41c4dfc8
Stoichiometry,enter_p + h_p --> enter_e + h_c Enterochelin + H+ --> Enterochelin + H+
GPR,CIW80_09625
Lower bound,0.0
Upper bound,1000.0


In [40]:
modelCopy.reactions.FEENTERtex_copy1

Reaction identifier,FEENTERtex_copy1
Name,Enterochelin transport (secretion periplasm)
Memory address,0x01ed41b1a9c8
Stoichiometry,enter_p + h_p --> enter_e + h_c Enterochelin + H+ --> Enterochelin + H+
GPR,CIW80_09625
Lower bound,0.0
Upper bound,1000.0


In [41]:
modelCopy.reactions.FEENTERtex_copy2

Reaction identifier,FEENTERtex_copy2
Name,Enterochelin transport (secretion periplasm)
Memory address,0x01ed41b0cd48
Stoichiometry,enter_p + h_p --> enter_e + h_c Enterochelin + H+ --> Enterochelin + H+
GPR,CIW80_13505
Lower bound,0.0
Upper bound,1000.0


On Biocyc, CIW80_09625 is annotated as outer membrane channel protein TolC (tolC). https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS09575
<br>CIW80_13505 is annotated as a "phosphate ABC transporter periplasmic binding protein". https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS13390

The latter does not seem to be related to iron transport. Both copies will be removed.

In [42]:
# Remove the two copies
modelCopy.remove_reactions(['FEENTERtex_copy1', 'FEENTERtex_copy2'])

In [43]:
# Update in reaction overview
rxn_origin_df.loc['FEENTERtex_copy1', 'added'] = 'removed'
rxn_origin_df.loc['FEENTERtex_copy1', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

rxn_origin_df.loc['FEENTERtex_copy2', 'added'] = 'removed'
rxn_origin_df.loc['FEENTERtex_copy2', 'notes'] = 'Incorrect gene. Removed in 4.1. Original was kept.'

### GLCtex

There are three versions:
- Original: Bidirectional reaction
- Copy1: One-direction reaction (fwd), with different gene
- Copy2: Bidirectional reaction, with same genes as original

In [44]:
modelCopy.reactions.GLCtex

Reaction identifier,GLCtex
Name,Glucose transport via diffusion (extracellular to periplasm)
Memory address,0x01ed41b0c908
Stoichiometry,glc__D_e <=> glc__D_p D-Glucose <=> D-Glucose
GPR,CIW80_19480 or CIW80_22660 or CIW80_25725 or CIW80_05020
Lower bound,-1000.0
Upper bound,1000.0


In [45]:
modelCopy.reactions.GLCtex_copy1

Reaction identifier,GLCtex_copy1
Name,Glucose transport via diffusion (extracellular to periplasm)
Memory address,0x01ed41c62248
Stoichiometry,glc__D_e --> glc__D_p D-Glucose --> D-Glucose
GPR,CIW80_15320
Lower bound,0.0
Upper bound,1000.0


In [46]:
modelCopy.reactions.GLCtex_copy2

Reaction identifier,GLCtex_copy2
Name,Glucose transport via diffusion (extracellular to periplasm)
Memory address,0x01ed41bc5408
Stoichiometry,glc__D_e <=> glc__D_p D-Glucose <=> D-Glucose
GPR,CIW80_19480 or CIW80_22660 or CIW80_25725 or CIW80_05020
Lower bound,-1000.0
Upper bound,1000.0


CIW80_19480 outer membrane porin PhoE "_Induced by phosphate limitation, PhoE facilitates efficient diffusion of phosphate and phosphorus-containing compounds across the outer membrane_"https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS19290
<br> CIW80_22660 outer membrane porin F. "_OmpF is a general outer membrane (OM) porin which mediates the non-specific diffusion of small solutes such as sugars, ions and amino acids_" https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS22425
<br> CIW80_25725 outer membrane porin N. "_Though under typical laboratory growth conditions OmpN is not highly expressed, when overexpressed it operates as a porin with single-channel conductance properties similar to those of OmpC_" https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS25485
<br> CIW80_05020 outer membrane porin C. "_OmpC is a general outer membrane (OM) porin which mediates the non-specific diffusion of small solutes such as sugars, ions and amino acids_" https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS05000
<br> 
<br> CIW80_15320 maltose outer membrane channel / phage lambda receptor protein. "_LamB is a member of the Sugar Porin (SP) family. It specifically facilitates the diffusion of maltose and other maltodextrins (α1-4 linked polyglucosyls) across the outer membrane_" https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS15170

The gene of the first copy facilitates the diffusion of maltose, not glucose. Therefore only the original reaction is kept

In [47]:
# Remove the two copies
modelCopy.remove_reactions(['GLCtex_copy1', 'GLCtex_copy2'])

In [48]:
# Update in reaction overview
rxn_origin_df.loc['GLCtex_copy1', 'added'] = 'removed'
rxn_origin_df.loc['GLCtex_copy1', 'notes'] = 'Incorrect gene. Removed in 4.1. Original was kept.'

rxn_origin_df.loc['GLCtex_copy2', 'added'] = 'removed'
rxn_origin_df.loc['GLCtex_copy2', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

### GLYBt2pp

There are three versions:
- Original: One-direction reaction (fwd)
- Copy1: One-direction reaction (fwd), same gene as original
- Copy2: Bi-directional reaction, same gene as original

In [49]:
modelCopy.reactions.GLYBt2pp

Reaction identifier,GLYBt2pp
Name,Glycine betaine transport via proton symport (periplasm)
Memory address,0x01ed41b82ec8
Stoichiometry,glyb_p + h_p --> glyb_c + h_c Glycine betaine + H+ --> Glycine betaine + H+
GPR,CIW80_15850
Lower bound,0.0
Upper bound,1000.0


In [50]:
modelCopy.reactions.GLYBt2pp_copy1

Reaction identifier,GLYBt2pp_copy1
Name,Glycine betaine transport via proton symport (periplasm)
Memory address,0x01ed41b23b48
Stoichiometry,glyb_p + h_p --> glyb_c + h_c Glycine betaine + H+ --> Glycine betaine + H+
GPR,CIW80_15850
Lower bound,0.0
Upper bound,1000.0


In [51]:
modelCopy.reactions.GLYBt2pp_copy2

Reaction identifier,GLYBt2pp_copy2
Name,Glycine betaine transport via proton symport (periplasm)
Memory address,0x01ed41b23c48
Stoichiometry,glyb_p + h_p <=> glyb_c + h_c Glycine betaine + H+ <=> Glycine betaine + H+
GPR,CIW80_15850
Lower bound,-1000.0
Upper bound,1000.0


Based on the name "transport IN" and the biocyc description of this process, it seems that this reaction should be one-directional (fwd). (https://biocyc.org/compound?orgid=GCF_003546975&id=BETAINE#tab=RXNS)

In [52]:
# Remove copy1 and copy2
modelCopy.remove_reactions(['GLYBt2pp_copy1', 'GLYBt2pp_copy2'])

In [53]:
# Update in reaction overview
rxn_origin_df.loc['GLYBt2pp_copy1', 'added'] = 'removed'
rxn_origin_df.loc['GLYBt2pp_copy1', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

rxn_origin_df.loc['GLYBt2pp_copy2', 'added'] = 'removed'
rxn_origin_df.loc['GLYBt2pp_copy2', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

### GTHPi

There are three versions:
- Original: One-direction reaction (fwd)
- Copy1: One-direction reaction (fwd), same gene as original
- Copy2: one-direction reaction, same gene as original

In [54]:
modelCopy.reactions.GTHPi

Reaction identifier,GTHPi
Name,Glutathione peridoxase
Memory address,0x01ed41b68988
Stoichiometry,2.0 gthrd_c + h2o2_c --> gthox_c + 2.0 h2o_c 2.0 Reduced glutathione + Hydrogen peroxide --> Oxidized glutathione + 2.0 H2O H2O
GPR,CIW80_01430
Lower bound,0.0
Upper bound,1000.0


In [55]:
modelCopy.reactions.GTHPi_copy1

Reaction identifier,GTHPi_copy1
Name,Glutathione peridoxase
Memory address,0x01ed41b38c48
Stoichiometry,2.0 gthrd_c + h2o2_c --> gthox_c + 2.0 h2o_c 2.0 Reduced glutathione + Hydrogen peroxide --> Oxidized glutathione + 2.0 H2O H2O
GPR,CIW80_01430
Lower bound,0.0
Upper bound,1000.0


In [56]:
modelCopy.reactions.GTHPi_copy2

Reaction identifier,GTHPi_copy2
Name,Glutathione peridoxase
Memory address,0x01ed41b38f88
Stoichiometry,2.0 gthrd_c + h2o2_c --> gthox_c + 2.0 h2o_c 2.0 Reduced glutathione + Hydrogen peroxide --> Oxidized glutathione + 2.0 H2O H2O
GPR,CIW80_01430
Lower bound,0.0
Upper bound,1000.0


In [57]:
# All the same. Remove copy1 and copy2
modelCopy.remove_reactions(['GTHPi_copy1', 'GTHPi_copy2'])

In [58]:
# Update in reaction overview
rxn_origin_df.loc['GTHPi_copy1', 'added'] = 'removed'
rxn_origin_df.loc['GTHPi_copy1', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

rxn_origin_df.loc['GTHPi_copy2', 'added'] = 'removed'
rxn_origin_df.loc['GTHPi_copy2', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

### ICHORS

There are three versions:
- Original: Bi-directional reaction
- Copy1: One-direction reaction (fwd), with different gene
- Copy2: Bi-directional reaction, with same genes as original

In [59]:
modelCopy.reactions.ICHORS

Reaction identifier,ICHORS
Name,Isochorismate synthase
Memory address,0x01ed41b0ccc8
Stoichiometry,chor_c <=> ichor_c Chorismate <=> Isochorismate
GPR,CIW80_05275
Lower bound,-1000.0
Upper bound,1000.0


In [60]:
modelCopy.reactions.ICHORS_copy1

Reaction identifier,ICHORS_copy1
Name,Isochorismate synthase
Memory address,0x01ed41cd7408
Stoichiometry,chor_c --> ichor_c Chorismate --> Isochorismate
GPR,CIW80_20975
Lower bound,0.0
Upper bound,1000.0


In [61]:
modelCopy.reactions.ICHORS_copy2

Reaction identifier,ICHORS_copy2
Name,Isochorismate synthase
Memory address,0x01ed41ccbd88
Stoichiometry,chor_c <=> ichor_c Chorismate <=> Isochorismate
GPR,CIW80_05275
Lower bound,-1000.0
Upper bound,1000.0


CIW80_05275: isochorismate synthase MenF. https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS05245
<br> CIW80_20975 isochorismate synthase EntC. https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS20755

On BioCyc the first reaction is favoured in the forward direction, the other is reversible. Therefore the bi-directional reaction will be kept with both genes in the GPR.

In [62]:
# Change GPR of original reaction
modelCopy.reactions.ICHORS.gene_reaction_rule = 'CIW80_05275 or CIW80_20975'

# Remove both copies
modelCopy.remove_reactions(['ICHORS_copy1', 'ICHORS_copy2'])

In [63]:
# Update in reaction overview
rxn_origin_df.loc['ICHORS_copy1', 'added'] = 'removed'
rxn_origin_df.loc['ICHORS_copy1', 'notes'] = 'Combined with original reaction "ICHORS". Removed in 4.1. Adapted original was kept.'

rxn_origin_df.loc['ICHORS_copy2', 'added'] = 'removed'
rxn_origin_df.loc['ICHORS_copy2', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

### PRASCSi

There are three versions:
- Original:  One-direction reaction (fwd)
- Copy1: One-direction reaction (fwd), with same gene
- Copy2: Bi-directional reaction, with same genes as original and copy1

In [64]:
modelCopy.reactions.PRASCSi

Reaction identifier,PRASCSi
Name,Phosphoribosylaminoimidazolesuccinocarboxamide synthase
Memory address,0x01ed41c63ac8
Stoichiometry,5aizc_c + asp__L_c + atp_c --> 25aics_c + adp_c + h_c + pi_c 5-amino-1-(5-phospho-D-ribosyl)imidazole-4-carboxylate + L-Aspartate + ATP C10H12N5O13P3 --> (S)-2-[5-Amino-1-(5-phospho-D-ribosyl)imidazole-4-carboxamido]succinate + ADP C10H12N5O10P2 + H+ +...
GPR,CIW80_06235
Lower bound,0.0
Upper bound,1000.0


In [65]:
modelCopy.reactions.PRASCSi_copy1

Reaction identifier,PRASCSi_copy1
Name,Phosphoribosylaminoimidazolesuccinocarboxamide synthase
Memory address,0x01ed41b2cf48
Stoichiometry,5aizc_c + asp__L_c + atp_c --> 25aics_c + adp_c + h_c + pi_c 5-amino-1-(5-phospho-D-ribosyl)imidazole-4-carboxylate + L-Aspartate + ATP C10H12N5O13P3 --> (S)-2-[5-Amino-1-(5-phospho-D-ribosyl)imidazole-4-carboxamido]succinate + ADP C10H12N5O10P2 + H+ +...
GPR,CIW80_06235
Lower bound,0.0
Upper bound,1000.0


In [66]:
modelCopy.reactions.PRASCSi_copy2

Reaction identifier,PRASCSi_copy2
Name,Phosphoribosylaminoimidazolesuccinocarboxamide synthase
Memory address,0x01ed41b2cc08
Stoichiometry,5aizc_c + asp__L_c + atp_c <=> 25aics_c + adp_c + h_c + pi_c 5-amino-1-(5-phospho-D-ribosyl)imidazole-4-carboxylate + L-Aspartate + ATP C10H12N5O13P3 <=> (S)-2-[5-Amino-1-(5-phospho-D-ribosyl)imidazole-4-carboxamido]succinate + ADP C10H12N5O10P2 + H+ +...
GPR,CIW80_06235
Lower bound,-1000.0
Upper bound,1000.0


On BioCyc the reaction is predicted to be favoured in the forward direction (https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS06195#tab=RXNS). Therefore only the original gene was be kept

In [67]:
# Remove both copies
modelCopy.remove_reactions(['PRASCSi_copy1', 'PRASCSi_copy2'])

In [68]:
# Update in reaction overview
rxn_origin_df.loc['PRASCSi_copy1', 'added'] = 'removed'
rxn_origin_df.loc['PRASCSi_copy1', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

rxn_origin_df.loc['PRASCSi_copy2', 'added'] = 'removed'
rxn_origin_df.loc['PRASCSi_copy2', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

### UAG4Ei

There are two versions:
- Copy1: One-direction reaction (fwd)
- Copy2: One-direction reaction (fwd), with same gene

In [69]:
modelCopy.reactions.UAG4Ei_copy1

Reaction identifier,UAG4Ei_copy1
Name,UDP-N-acetylglucosamine 4-epimerase
Memory address,0x01ed41b1a5c8
Stoichiometry,uacgam_c --> udpacgal_c UDP-N-acetyl-D-glucosamine --> UDP-N-acetyl-D-galactosamine
GPR,CIW80_03965
Lower bound,0.0
Upper bound,1000.0


In [70]:
modelCopy.reactions.UAG4Ei_copy2

Reaction identifier,UAG4Ei_copy2
Name,UDP-N-acetylglucosamine 4-epimerase
Memory address,0x01ed41b1a708
Stoichiometry,uacgam_c --> udpacgal_c UDP-N-acetyl-D-glucosamine --> UDP-N-acetyl-D-galactosamine
GPR,CIW80_03965
Lower bound,0.0
Upper bound,1000.0


In [71]:
# Remove the double reaction
modelCopy.remove_reactions(['UAG4Ei_copy2'])

# Rename copy1
modelCopy.reactions.UAG4Ei_copy1.id = 'UAG4Ei'

In [72]:
# Update in reaction overview
rxn_origin_df.loc['UAG4Ei_copy1', 'notes'] = 'Renamed to UAG4Ei in 4.1.'

rxn_origin_df.loc['UAG4Ei_copy2', 'added'] = 'removed'
rxn_origin_df.loc['UAG4Ei_copy2', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

rxn_origin_df.loc['UAG4Ei', 'origin'] = rxn_origin_df.loc['UAG4Ei_copy1', 'origin']
rxn_origin_df.loc['UAG4Ei', 'added'] = rxn_origin_df.loc['UAG4Ei_copy1', 'added']
rxn_origin_df.loc['UAG4Ei', 'notebook'] = rxn_origin_df.loc['UAG4Ei_copy1', 'notebook']
rxn_origin_df.loc['UAG4Ei', 'notes'] = 'UAG4Ei_copy1 was renamed to UAG4Ei in 4.1.'

### URAt2pp

There are three versions:
- Original:  One-direction reaction (fwd) with two genes
- Copy1: One-direction reaction (fwd), with one of the genes from original
- Copy2: Bi-directional reaction, with different genes from original and copy1

In [73]:
modelCopy.reactions.URAt2pp

Reaction identifier,URAt2pp
Name,Uracil transport in via proton symport (periplasm)
Memory address,0x01ed41bc1a48
Stoichiometry,h_p + ura_p --> h_c + ura_c H+ + Uracil --> H+ + Uracil
GPR,CIW80_06280 or CIW80_22995
Lower bound,0.0
Upper bound,1000.0


In [74]:
modelCopy.reactions.URAt2pp_copy1

Reaction identifier,URAt2pp_copy1
Name,Uracil transport in via proton symport (periplasm)
Memory address,0x01ed41b2ccc8
Stoichiometry,h_p + ura_p --> h_c + ura_c H+ + Uracil --> H+ + Uracil
GPR,CIW80_06280
Lower bound,0.0
Upper bound,1000.0


In [75]:
modelCopy.reactions.URAt2pp_copy2

Reaction identifier,URAt2pp_copy2
Name,Uracil transport in via proton symport (periplasm)
Memory address,0x01ed41b1ad08
Stoichiometry,h_p + ura_p <=> h_c + ura_c H+ + Uracil <=> H+ + Uracil
GPR,CIW80_22995
Lower bound,-1000.0
Upper bound,1000.0


URAt2pp: Bi-directional (https://biocyc.org/compound?orgid=GCF_003546975&id=URACIL#tab=RXNS)

CIW80_06280: uracil:H+ symporter (uraA). https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS06240
<br> CIW80_22995: pyrimidine:H+ symporter (rutG). https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS22760
Both can be related to uracil transport. Keep both genes with the bi-directional reaction

In [76]:
# Remove the one-directional reactions
modelCopy.remove_reactions(['URAt2pp', 'URAt2pp_copy1'])

# Rename the bi-directional reactions
modelCopy.reactions.URAt2pp_copy2.id = 'URAt2pp'

# Update the GPR
modelCopy.reactions.URAt2pp.gene_reaction_rule = 'CIW80_06280 or CIW80_22995'

In [77]:
# "delete" the incorrect genes
for rxn in ['URAt2pp', 'URAt2pp_copy1']:
    rxn_origin_df.loc[rxn, 'added'] = 'removed'
    rxn_origin_df.loc[rxn, 'notes'] = 'Removed in 4.1."%s_copy2" was kept as correct reaction.'%rxn
    
# Update the note of "_copy2"
rxn_origin_df.loc['URAt2pp_copy2', 'notes'] = 'Renamed to %s in 4.1. Original reaction and "URAt2pp_copy1" were removed'

### VPAMTr

There are three versions:
- Original:  Bi-directional reaction
- Copy1: Bi-directional reaction, with same gene as original
- Copy2: One-direction reaction (fwd), with same gene as original and copy1

In [78]:
modelCopy.reactions.VPAMTr

Reaction identifier,VPAMTr
Name,Valine-pyruvate aminotransferase
Memory address,0x01ed41b52448
Stoichiometry,3mob_c + ala__L_c <=> pyr_c + val__L_c 3-Methyl-2-oxobutanoate + L-Alanine <=> Pyruvate + L-Valine
GPR,CIW80_12580
Lower bound,-1000.0
Upper bound,1000.0


In [79]:
modelCopy.reactions.VPAMTr_copy1

Reaction identifier,VPAMTr_copy1
Name,Valine-pyruvate aminotransferase
Memory address,0x01ed41b2c208
Stoichiometry,3mob_c + ala__L_c <=> pyr_c + val__L_c 3-Methyl-2-oxobutanoate + L-Alanine <=> Pyruvate + L-Valine
GPR,CIW80_12580
Lower bound,-1000.0
Upper bound,1000.0


In [80]:
modelCopy.reactions.VPAMTr_copy2

Reaction identifier,VPAMTr_copy2
Name,Valine-pyruvate aminotransferase
Memory address,0x01ed41b2c0c8
Stoichiometry,3mob_c + ala__L_c --> pyr_c + val__L_c 3-Methyl-2-oxobutanoate + L-Alanine --> Pyruvate + L-Valine
GPR,CIW80_12580
Lower bound,0.0
Upper bound,1000.0


This reaction is part of the alanin biosynthesis pathway and is described as a reversible reaction (https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS12475#tab=RXNS). Therefore the bi-directional reaction was kept.

In [81]:
# Remove copy1 and copy2
modelCopy.remove_reactions(['VPAMTr_copy1', 'VPAMTr_copy2'])

In [82]:
# Update in reaction overview
rxn_origin_df.loc['VPAMTr_copy1', 'added'] = 'removed'
rxn_origin_df.loc['VPAMTr_copy1', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

rxn_origin_df.loc['VPAMTr_copy2', 'added'] = 'removed'
rxn_origin_df.loc['VPAMTr_copy2', 'notes'] = 'Duplicate reaction. Removed in 4.1. Original was kept.'

### NTP1 & ATPM

In [83]:
modelCopy.reactions.NTP1

Reaction identifier,NTP1
Name,Nucleoside-triphosphatase (ATP)
Memory address,0x01ed41bfa2c8
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP C10H12N5O13P3 + H2O H2O --> ADP C10H12N5O10P2 + H+ + Phosphate
GPR,CIW80_16125
Lower bound,0.0
Upper bound,1000.0


In [84]:
modelCopy.reactions.ATPM

Reaction identifier,ATPM
Name,ATP maintenance requirement
Memory address,0x01ed41c15288
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP C10H12N5O13P3 + H2O H2O --> ADP C10H12N5O10P2 + H+ + Phosphate
GPR,
Lower bound,6.86
Upper bound,1000.0


Both reactions for the ATP reaction are also present in MG1655 model and were therefore both kept

# 4. Identify different versions of the same reaction

In [85]:
# In addition to copies, there are also different versions of reactions used in different models
# Therefore, additionally find reactions with have the appendix "_1", "_2", and "_3"
gene_copies_1 = reaction_df[reaction_df['ID'].str.contains("_1")].sort_values('ID')
gene_copies_1 = gene_copies_1[~gene_copies_1['ID'].str.contains("EX")].sort_values('ID')
gene_copies_1 = gene_copies_1[~gene_copies_1['ID'].str.contains("1p")].sort_values('ID')
gene_copies_1

,ID,formula,compartment,genes
2814,3NTD2pp_1,3dhguln_c + 3ump_p + atp_c + h2o_p --> 3dhguln...,"{c, p}",(CIW80_16440)
2615,CELLBpts_1,cellb_e + pep_c --> 6pgg_c + pyr_p,"{e, c, p}","(CIW80_00635, CIW80_01555, CIW80_00630)"
2805,MNabc_1,atp_c + h2o_c + mn2_p --> adp_c + h_c + mn2_c ...,"{c, p}","(CIW80_24590, CIW80_25715, CIW80_24595, CIW80_..."
2816,MTHFD_1,mlthf_c + nadp_c <=> h_c + methf_c + nadph_c,{c},(CIW80_20780)
2817,QULNS_1,dhap_c + iasp_c --> 2.0 h2o_c + h_c + pi_c + q...,{c},(CIW80_21725)


In [86]:
# "_2" reactions
gene_copies_2 = reaction_df[reaction_df['ID'].str.contains("_2")].sort_values('ID')
gene_copies_2 = gene_copies_2[~gene_copies_2['ID'].str.contains("EX")].sort_values('ID')
gene_copies_2 = gene_copies_2[~gene_copies_2['ID'].str.contains("2pp")].sort_values('ID')
gene_copies_2

,ID,formula,compartment,genes
2739,AMMQT8_2,2dmmq8_c + amet_c --> ahcys_c + h_c + mqn8_c,{c},(CIW80_14155)
2818,CYTK2_2,atp_c + ctp_c + dcmp_c <=> adp_c + cdp_c + dcdp_c,{c},(CIW80_22565)
2815,GLYOX_2,h2o_c + lgt__S_c --> gthrd_c + h_c + lac__D_c ...,{c},(CIW80_18840)
311,PFK_2,atp_c + tag6p__D_c --> adp_c + h_c + tagdp__D_c,{c},(CIW80_14585)


In [87]:
# "_3" reactions
gene_copies_3 = reaction_df[reaction_df['ID'].str.contains("_3")].sort_values('ID')
gene_copies_3 = gene_copies_3[~gene_copies_3['ID'].str.contains("EX")].sort_values('ID')
gene_copies_3 = gene_copies_3[~gene_copies_3['ID'].str.contains("3pp")].sort_values('ID')
gene_copies_3

,ID,formula,compartment,genes
8,PFK_3,atp_c + s7p_c --> adp_c + h_c + s17bp_c,{c},(CIW80_14585)


# 5. Check reactions (different versions)

###  3NTD2pp_1
This reaction is only present in the model "iECIAI1_1343"

In [88]:
modelCopy.reactions.get_by_id('3NTD2pp_1')

Reaction identifier,3NTD2pp_1
Name,3'-nucleotidase (UMP) (periplasm)
Memory address,0x01ed41b1a588
Stoichiometry,3dhguln_c + 3ump_p + atp_c + h2o_p --> 3dhgulnp_c + adp_c + h_c + pi_p + uri_p 3-Dehydro-L-gulonate + 3 UMP C9H11N2O9P + ATP C10H12N5O13P3 + H2O H2O --> 3-keto-L-gulonate-6-phosphate + ADP C10H12N5O10P2 + H+ + Phosphate + Uridine
GPR,CIW80_16440
Lower bound,0.0
Upper bound,1000.0


In [89]:
modelCopy.reactions.get_by_id('3NTD2pp')

Reaction identifier,3NTD2pp
Name,3'-nucleotidase (UMP) (periplasm)
Memory address,0x01ed41c15ac8
Stoichiometry,3ump_p + h2o_p --> pi_p + uri_p 3 UMP C9H11N2O9P + H2O H2O --> Phosphate + Uridine
GPR,CIW80_16440 or CIW80_20025
Lower bound,0.0
Upper bound,1000.0


No evidence could be found in BioCyc and literature for the reaction '3NTD2pp_1' in Nissle, which will therefore be removed

In [90]:
# Remove the duplicate
modelCopy.remove_reactions(['3NTD2pp_1'])

In [91]:
# Update in reaction overview
rxn_origin_df.loc['3NTD2pp_1', 'added'] = 'removed'
rxn_origin_df.loc['3NTD2pp_1', 'notes'] = 'No evidence for presence in EcN. Removed in 4.1. 3NTD2pp was kept.'

### CELLBpts_1
In the existing models the reactions are not both present in the same model. In both reactions the same metabolites are converted, but the location is different. In 'CELLBpts' extracellular cellobiose is transported and converted by using a periplasmic phosphoenolpyruvate (PEP). In 'CELLBpts_1' cytosolic PEP is used. In the mechanism of the PEP:Pyr PTS the phosphoryl group of PEP is transferred to EI of the transporter complex, thereby converting it to Pyr. So the correct reaction here is CELLBpts, in which periplasmid PEP is converted.

In [92]:
modelCopy.reactions.get_by_id('CELLBpts_1')

Reaction identifier,CELLBpts_1
Name,Cellobiose transport via PEP:Pyr PTS
Memory address,0x01ed41b38d88
Stoichiometry,"cellb_e + pep_c --> 6pgg_c + pyr_p Cellobiose + Phosphoenolpyruvate --> 6-Phospho-beta-D-glucosyl-(1,4)-D-glucose + Pyruvate"
GPR,CIW80_00635 or (CIW80_01555 and CIW80_00630)
Lower bound,0.0
Upper bound,1000.0


In [93]:
modelCopy.reactions.get_by_id('CELLBpts')

Reaction identifier,CELLBpts
Name,Cellobiose transport via PEP:Pyr PTS
Memory address,0x01ed41b0cd08
Stoichiometry,"cellb_e + pep_p --> 6pgg_c + pyr_p Cellobiose + Phosphoenolpyruvate --> 6-Phospho-beta-D-glucosyl-(1,4)-D-glucose + Pyruvate"
GPR,CIW80_00635
Lower bound,0.0
Upper bound,1000.0


In [94]:
# Remove the duplicate
modelCopy.remove_reactions(['CELLBpts_1'])

In [95]:
# Update in reaction overview
rxn_origin_df.loc['CELLBpts_1', 'added'] = 'removed'
rxn_origin_df.loc['CELLBpts_1', 'notes'] = 'Incorrect location of metabolites. Removed in 4.1. CELLBpts was kept.'

### MNabc_1, AMMQT8_2
No other version present in the model

### MTHFD_1

There are two versions:
- Original:  Bi-directional reaction
- Copy1: Bi-directional reaction, with same gene as original and an additional proton

In [96]:
modelCopy.reactions.MTHFD

Reaction identifier,MTHFD
Name,Methylenetetrahydrofolate dehydrogenase (NADP)
Memory address,0x01ed41c56c08
Stoichiometry,"mlthf_c + nadp_c <=> methf_c + nadph_c 5,10-Methylenetetrahydrofolate + Nicotinamide adenine dinucleotide phosphate <=> 5,10-Methenyltetrahydrofolate + Nicotinamide adenine dinucleotide phosphate - reduced"
GPR,CIW80_20780
Lower bound,-1000.0
Upper bound,1000.0


In [97]:
modelCopy.reactions.MTHFD_1

Reaction identifier,MTHFD_1
Name,Methylenetetrahydrofolate dehydrogenase (NADP)
Memory address,0x01ed41b1aa88
Stoichiometry,"mlthf_c + nadp_c <=> h_c + methf_c + nadph_c 5,10-Methylenetetrahydrofolate + Nicotinamide adenine dinucleotide phosphate <=> H+ + 5,10-Methenyltetrahydrofolate + Nicotinamide adenine dinucleotide phosphate - reduced"
GPR,CIW80_20780
Lower bound,-1000.0
Upper bound,1000.0


In [98]:
modelCopy.reactions.MTHFD_1.check_mass_balance()

{'charge': 1.0, 'H': 1.0}

The additional proton should not be present. Therefore only MTHFD was kept

In [99]:
# Remove the duplicate
modelCopy.remove_reactions(['MTHFD_1'])

In [100]:
# Update in reaction overview
rxn_origin_df.loc['MTHFD_1', 'added'] = 'removed'
rxn_origin_df.loc['MTHFD_1', 'notes'] = 'Incorrect mass balance. Removed in 4.1. MTHFD was kept.'

### QULNS_1

There are two versions:
- Original:  One-directional reaction
- Copy1: One-directional reaction, with same gene as original and additional proton

In [101]:
modelCopy.reactions.QULNS

Reaction identifier,QULNS
Name,Quinolinate synthase
Memory address,0x01ed41c56dc8
Stoichiometry,dhap_c + iasp_c --> 2.0 h2o_c + pi_c + quln_c Dihydroxyacetone phosphate + Iminoaspartate --> 2.0 H2O H2O + Phosphate + Quinolinate
GPR,CIW80_21725
Lower bound,0.0
Upper bound,1000.0


In [102]:
modelCopy.reactions.QULNS_1

Reaction identifier,QULNS_1
Name,Quinolinate synthase
Memory address,0x01ed41b1a7c8
Stoichiometry,dhap_c + iasp_c --> 2.0 h2o_c + h_c + pi_c + quln_c Dihydroxyacetone phosphate + Iminoaspartate --> 2.0 H2O H2O + H+ + Phosphate + Quinolinate
GPR,CIW80_21725
Lower bound,0.0
Upper bound,1000.0


In [103]:
modelCopy.reactions.QULNS_1.check_mass_balance()

{'charge': 1.0, 'H': 1.0}

The additional proton should not be present. Therefore only QULNS was kept

In [104]:
# Remove the duplicate
modelCopy.remove_reactions(['QULNS_1'])

In [105]:
# Update in reaction overview
rxn_origin_df.loc['QULNS_1', 'added'] = 'removed'
rxn_origin_df.loc['QULNS_1', 'notes'] = 'Incorrect mass balance. Removed in 4.1. QULNS was kept.'

### CYTK2_2
This reaction is only present in the model "iECIAI1_1343"

In [106]:
modelCopy.reactions.get_by_id('CYTK2_2')

Reaction identifier,CYTK2_2
Name,Cytidylate kinase (dCMP)
Memory address,0x01ed41b1a888
Stoichiometry,atp_c + ctp_c + dcmp_c <=> adp_c + cdp_c + dcdp_c ATP C10H12N5O13P3 + CTP C9H12N3O14P3 + DCMP C9H12N3O7P <=> ADP C10H12N5O10P2 + CDP C9H12N3O11P2 + DCDP C9H12N3O10P2
GPR,CIW80_22565
Lower bound,-1000.0
Upper bound,1000.0


In [107]:
modelCopy.reactions.get_by_id('CYTK2')

Reaction identifier,CYTK2
Name,Cytidylate kinase (dCMP)
Memory address,0x01ed41cb6e08
Stoichiometry,atp_c + dcmp_c <=> adp_c + dcdp_c ATP C10H12N5O13P3 + DCMP C9H12N3O7P <=> ADP C10H12N5O10P2 + DCDP C9H12N3O10P2
GPR,CIW80_22565
Lower bound,-1000.0
Upper bound,1000.0


The gene CIW80_22565 (cmk) was predicted to be present in EcN https://biocyc.org/gene?orgid=GCF_003546975&id=CIW80_RS22325. The reaction that is catalysed by the product of cmk is CYTK2. Therefore CYTK2_2 will be removed.

In [108]:
# Remove the duplicate
modelCopy.remove_reactions(['CYTK2_2'])

In [109]:
# Update in reaction overview
rxn_origin_df.loc['CYTK2_2', 'added'] = 'removed'
rxn_origin_df.loc['CYTK2_2', 'notes'] = 'Incorrect reaction. Removed in 4.1. CYTK2 was kept.'

### GLYOX_2
In the reaction 'GLYOX_2', the metabolite 'mthgxl_c' is added to the original reaction. However, based on the reaction balance it seems that this is a mistake. Therefore GLYOX_2 will be removed.

In [110]:
modelCopy.reactions.get_by_id('GLYOX_2')

Reaction identifier,GLYOX_2
Name,Hydroxyacylglutathione hydrolase
Memory address,0x01ed41b1a948
Stoichiometry,h2o_c + lgt__S_c --> gthrd_c + h_c + lac__D_c + mthgxl_c H2O H2O + (R)-S-Lactoylglutathione --> Reduced glutathione + H+ + D-Lactate + Methylglyoxal
GPR,CIW80_18840
Lower bound,0.0
Upper bound,1000.0


In [111]:
modelCopy.reactions.get_by_id('GLYOX_2').check_mass_balance()

{'C': 3.0, 'H': 4.0, 'O': 2.0}

In [112]:
modelCopy.reactions.get_by_id('GLYOX')

Reaction identifier,GLYOX
Name,Hydroxyacylglutathione hydrolase
Memory address,0x01ed41c15488
Stoichiometry,h2o_c + lgt__S_c --> gthrd_c + h_c + lac__D_c H2O H2O + (R)-S-Lactoylglutathione --> Reduced glutathione + H+ + D-Lactate
GPR,CIW80_18840
Lower bound,0.0
Upper bound,1000.0


In [113]:
# Remove the duplicate
modelCopy.remove_reactions(['GLYOX_2'])

In [114]:
# Update in reaction overview
rxn_origin_df.loc['GLYOX_2', 'added'] = 'removed'
rxn_origin_df.loc['GLYOX_2', 'notes'] = 'Incorrect mass balance. Removed in 4.1. GLYOX was kept.'

### PFK_2 & PFK_3
Promiscuous enzyme that can catalyse multiple sugars. Keep all three reactions (PFK, PFK_2, PFK_3)

In [115]:
modelCopy.reactions.PFK

Reaction identifier,PFK
Name,Phosphofructokinase
Memory address,0x01ed41bfa148
Stoichiometry,"atp_c + f6p_c --> adp_c + fdp_c + h_c ATP C10H12N5O13P3 + D-Fructose 6-phosphate --> ADP C10H12N5O10P2 + D-Fructose 1,6-bisphosphate + H+"
GPR,CIW80_14585 or CIW80_01495
Lower bound,0.0
Upper bound,1000.0


In [116]:
modelCopy.reactions.get_by_id('PFK_2').reaction

'atp_c + tag6p__D_c --> adp_c + h_c + tagdp__D_c'

In [117]:
modelCopy.reactions.get_by_id('PFK_3').reaction

'atp_c + s7p_c --> adp_c + h_c + s17bp_c'

In [118]:
modelCopy.reactions.get_by_id('PFK').reaction

'atp_c + f6p_c --> adp_c + fdp_c + h_c'

# 6. Test model

In [119]:
#Establish a definition that initializes models to an in silico representation of M9 media

def m9(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0 
            
    model.reactions.EX_ca2_e.lower_bound=-1000
    model.reactions.EX_cl_e.lower_bound=-1000
    model.reactions.EX_co2_e.lower_bound=-1000
    model.reactions.EX_cobalt2_e.lower_bound=-1000
    model.reactions.EX_cu2_e.lower_bound=-1000
    model.reactions.EX_fe2_e.lower_bound=-1000
    model.reactions.EX_fe3_e.lower_bound=-1000
    model.reactions.EX_h_e.lower_bound=-1000
    model.reactions.EX_h2o_e.lower_bound=-1000
    model.reactions.EX_k_e.lower_bound=-1000
    model.reactions.EX_mg2_e.lower_bound=-1000
    model.reactions.EX_mn2_e.lower_bound=-1000
    model.reactions.EX_mobd_e.lower_bound=-1000
    model.reactions.EX_na1_e.lower_bound=-1000
    model.reactions.EX_tungs_e.lower_bound=-1000
    model.reactions.EX_zn2_e.lower_bound=-1000
    model.reactions.EX_ni2_e.lower_bound=-1000
    model.reactions.EX_sel_e.lower_bound=-1000
    model.reactions.EX_slnt_e.lower_bound=-1000
    model.reactions.EX_glc__D_e.lower_bound=-20
    model.reactions.EX_so4_e.lower_bound=-1000
    model.reactions.EX_nh4_e.lower_bound=-1000
    model.reactions.EX_pi_e.lower_bound=-1000
    model.reactions.EX_cbl1_e.lower_bound=-.01
    model.reactions.EX_o2_e.lower_bound=-20
       
    return model

In [120]:
# Test whether the EcN model can grown on glucose (EX_glc__D_e)
with modelCopy:
    m9(modelCopy)
    solution = modelCopy.optimize()
    
solution

,fluxes,reduced_costs
ALATA_D2,0.000000,-7.604370e-03
SHCHD2,0.000602,5.551115e-17
CPPPGO,0.000602,0.000000e+00
GTHOr,0.000000,-6.829559e-17
DHORD5,0.000000,-1.420500e-16
...,...,...
12PPDStpp,0.000000,0.000000e+00
MALt5,0.000000,3.453838e-17
SALCHS2FEexs,0.000000,6.199542e-19
SALCHS4FEexs,0.000000,6.199542e-19


# 7. Remaining duplicates

In [121]:
# Double check which reactions still have copies
reaction_df = pd.DataFrame(columns=['ID','formula', 'compartment', 'genes'])

for reaction in modelCopy.reactions:
    formula = reaction.reaction
    ID = reaction.id
    compartment = reaction.compartments
    genes = reaction.genes
    data = {'ID': ID, 'formula': formula, 'compartment': compartment, 'genes': genes}
    reaction_df = reaction_df.append(data, ignore_index=True)

gene_copies = reaction_df[reaction_df['ID'].str.contains("copy")].sort_values('ID')
gene_copies

,ID,formula,compartment,genes


All duplicate reactions seem to have been removed.

# 8. Check other causes elevated growth rate

In [122]:
rxn_df = pd.DataFrame(columns=['reaction', 'objective_value'])
rxn_df.set_index('reaction', inplace=True)
m9(modelCopy)

for rxn in reaction_df.ID.values.tolist():
    with modelCopy:
        modelCopy.remove_reactions(rxn)
        rxn_df.loc[rxn, 'objective_value'] = modelCopy.slim_optimize()
        
rxn_df

C:\Users\maxvan\AppData\Local\Continuum\anaconda3\envs\EcN_model\lib\site-packages\cobra\core\model.py:745: UserWarning: need to pass in a list
  warn("need to pass in a list")


,objective_value
reaction,
ALATA_D2,2.701498
SHCHD2,0.0
CPPPGO,2.701384
GTHOr,2.701498
DHORD5,2.701498
...,...
12PPDStpp,2.701498
MALt5,2.701498
SALCHS2FEexs,2.701498


In [123]:
growth_reduction = rxn_df[rxn_df['objective_value'] < 2.5]
growth_reduction = growth_reduction[growth_reduction['objective_value'] > 1]
growth_reduction

,objective_value
reaction,
EX_h_e,1.223074
Htex,1.223074
H2Otex,1.665348
H2Otpp,1.665348
EX_h2o_e,1.665348
YUMPS,1.12128
PSUDS,1.12128


In [124]:
modelCopy.reactions.YUMPS

Reaction identifier,YUMPS
Name,YUMP synthetase
Memory address,0x01ed41bfa508
Stoichiometry,r5p_c + ura_c <=> h2o_c + psd5p_c Alpha-D-Ribose 5-phosphate + Uracil <=> H2O H2O + Pseudouridine 5 phosphate C9H11N2O9P
GPR,CIW80_04645
Lower bound,-1000.0
Upper bound,1000.0


In [125]:
modelCopy.reactions.PSUDS

Reaction identifier,PSUDS
Name,Pseudouridylate synthase
Memory address,0x01ed41b2cac8
Stoichiometry,2.0 h_c + r5p_c + ura_c --> h2o_c + psd5p_c 2.0 H+ + Alpha-D-Ribose 5-phosphate + Uracil --> H2O H2O + Pseudouridine 5 phosphate C9H11N2O9P
GPR,CIW80_06805
Lower bound,0.0
Upper bound,1000.0


In [126]:
modelCopy.reactions.PSUDS.check_mass_balance()

{'charge': -2.0, 'H': -2.0}

These two reactions both describe the conversion of alpha-D-Ribose 5- phosphate. However, in the second reaction (PSUDS) additionally two protons are consumed. This is not correct based on the mass balance. Therefore the second reaction will be removed.

In [127]:
# Remove the duplicate
modelCopy.remove_reactions(['PSUDS'])

In [128]:
# Update in reaction overview
rxn_origin_df.loc['PSUDS', 'added'] = 'removed'
rxn_origin_df.loc['PSUDS', 'notes'] = 'Incorrect mass balance. Removed in 4.1. YUMPS was kept.'

# 9. Unbound flux cycles
Take the reactions with unbound flux from MEMOTE. Remove the reactions already present in iML1515 and check the remaining.

In [129]:
EcN_flux = ["ALATA_L","PPM","VALTA","HACD5","HACD6","HACD4","HACD7","ADK1","HACD1","HACD2","FADRx","NDPK1","ACCOAL","HPYRRx","PTA2","PPAKr","ADK3","SUCFUMtpp","ARGAGMt7pp","ABUTt2pp","ACt2rpp","TRSARr","NAtex","CRNt8pp","CRNt2rpp","CRNDt2rpp","INDOLEt2pp","R1PK","R15BPK","PROt2rpp","PROt4pp","GLUABUTt7pp","GLUt2rpp","GLUt4pp","GLBRAN2","GLYCLTt4pp","PTRCORNt7pp","HACD3","GLYCLTt2rpp","ARGORNt7pp","MALtex","HPYRI","NAt3pp","ACOAD7f","ACOAD5f","ACOAD3f","ACOAD4f","ACOAD6f","CAt6pp","CA2t3pp","MN2tpp","GLDBRAN2","HACD8","THFAT","FOMETRi","SUCASPtpp","ACOAD1fr","LEUt2rpp","INDOLEt2rpp","MN2tipp","ASPtpp","FUMt1pp","SUCCt1pp","DMALRED","HACD8i","PPCSCT","LEUt4rpp","AGMPTRCtpp","ACOAD2","HHDDI","OHEDH","2HH24DDH1_copy2","2HH24DDH1_copy1","MALt5"]
iML_flux = ["ALATA_L","PPM","VALTA","EX_fe2_e","ICHORS_copy1","ICHORS_copy2","ADK1","NDPK1","ACCOAL","HPYRRx","PTA2","PPAKr","ADK3","SUCFUMtpp","THMDt2pp_copy1","THMDt2pp_copy2","URIt2pp_copy1","URIt2pp_copy2","ABUTt2pp","ACt2rpp","ACt4pp","ADNt2pp_copy1","ADNt2pp_copy2","CYTDt2pp_copy1","CYTDt2pp_copy2","GLCtex_copy1","TRSARr","FE2tex","CRNt8pp","CRNt2rpp","CRNDt2rpp","INDOLEt2pp","R1PK","R15BPK","PROt2rpp","PROt4pp","GLUABUTt7pp","GLUt2rpp","GLUt4pp","GLBRAN2","GLYCLTt4pp","GLYCLTt2rpp","SUCOAS","HPYRI","NAt3pp","CAt6pp","CA2t3pp","MN2tpp","GLDBRAN2","THFAT","FOMETRi","SUCASPtpp","ACOAD1fr","INDOLEt2rpp","MN2tipp","ASPtpp","FUMt1pp","SUCCt1pp"]

In [130]:
flux_list = list(set(EcN_flux) - set(iML_flux))
flux_list

['HACD6',
 'NAtex',
 'ACOAD5f',
 'HACD4',
 'HACD3',
 'LEUt2rpp',
 'HACD8',
 'HACD7',
 'ACOAD4f',
 'MALt5',
 'ACOAD3f',
 'HACD1',
 '2HH24DDH1_copy1',
 'HHDDI',
 'AGMPTRCtpp',
 'ARGORNt7pp',
 'HACD8i',
 'PPCSCT',
 'LEUt4rpp',
 'ACOAD2',
 'PTRCORNt7pp',
 'OHEDH',
 'MALtex',
 'FADRx',
 'HACD2',
 'HACD5',
 'ARGAGMt7pp',
 '2HH24DDH1_copy2',
 'ACOAD7f',
 'DMALRED',
 'ACOAD6f']

### HACDx & HACDxi

In [131]:
modelCopy.reactions.get_by_id('HACD8')

Reaction identifier,HACD8
Name,"3-hydroxyacyl-CoA dehydrogenase (3-oxooctadecanoyl-CoA), peroxisomal"
Memory address,0x01ed41b7d608
Stoichiometry,3ohodcoa_c + h_c + nadh_c <=> 3hodcoa_c + nad_c 3-Oxooctadecanoyl-CoA + H+ + Nicotinamide adenine dinucleotide - reduced <=> (S)-3-Hydroxyoctadecanoyl-CoA + Nicotinamide adenine dinucleotide
GPR,CIW80_14210 or CIW80_05635
Lower bound,-1000.0
Upper bound,1000.0


In [132]:
modelCopy.reactions.get_by_id('HACD8i')

Reaction identifier,HACD8i
Name,3-hydroxyacyl-CoA dehydrogenase (3-oxooctadecanoyl-CoA)
Memory address,0x01ed41b2c688
Stoichiometry,3hodcoa_c + nad_c --> 3ohodcoa_c + h_c + nadh_c (S)-3-Hydroxyoctadecanoyl-CoA + Nicotinamide adenine dinucleotide --> 3-Oxooctadecanoyl-CoA + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,CIW80_14210 or CIW80_05635
Lower bound,0.0
Upper bound,1000.0


In [133]:
modelCopy.metabolites.get_by_id('3hodcoa_c').summary()

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


This is the case for all reactions of HACD1-8. Remove the reverse reaction (HACDxi) in all cases to prevent energy generating cycles

In [134]:
# Remove the reverse reactions
modelCopy.remove_reactions(['HACD1i','HACD2i','HACD3i','HACD4i','HACD5i','HACD6i','HACD7i', 'HACD8i'])

# Edit in reaction overview
for rxn in ['HACD1i','HACD2i','HACD3i','HACD4i','HACD5i','HACD6i','HACD7i', 'HACD8i']:
    rxn_origin_df.loc[rxn, 'added'] = 'removed'
    rxn_origin_df.loc[rxn, 'notes'] = 'Caused unbound flux. Removed in 4.1. "HACDx" (without i) was kept as correct reaction.'

### ACOAD & ACOADxf

In [135]:
modelCopy.reactions.ACOAD2

Reaction identifier,ACOAD2
Name,Acyl-CoA dehydrogenase (hexanoyl-CoA)
Memory address,0x01ed41b1ae88
Stoichiometry,h_c + hx2coa_c + nadh_c --> hxcoa_c + nad_c H+ + Trans-Hex-2-enoyl-CoA + Nicotinamide adenine dinucleotide - reduced --> Hexanoyl-CoA (n-C6:0CoA) + Nicotinamide adenine dinucleotide
GPR,CIW80_19405
Lower bound,0.0
Upper bound,1000.0


In [136]:
modelCopy.reactions.ACOAD2f

Reaction identifier,ACOAD2f
Name,Acyl-CoA dehydrogenase (hexanoyl-CoA)
Memory address,0x01ed41b82288
Stoichiometry,fad_c + hxcoa_c <=> fadh2_c + hx2coa_c Flavin adenine dinucleotide oxidized + Hexanoyl-CoA (n-C6:0CoA) <=> Flavin adenine dinucleotide reduced + Trans-Hex-2-enoyl-CoA
GPR,CIW80_19405
Lower bound,-1000.0
Upper bound,1000.0


In [137]:
modelCopy.reactions.get_by_id('FADRx')

Reaction identifier,FADRx
Name,FAD reductase
Memory address,0x01ed41cb6648
Stoichiometry,fad_c + h_c + nadh_c --> fadh2_c + nad_c Flavin adenine dinucleotide oxidized + H+ + Nicotinamide adenine dinucleotide - reduced --> Flavin adenine dinucleotide reduced + Nicotinamide adenine dinucleotide
GPR,CIW80_14200
Lower bound,0.0
Upper bound,1000.0


In [138]:
modelCopy.metabolites.hxcoa_c.summary()

Percent,Flux,Reaction,Definition
100.00%,0.399,ACOAD2,h_c + hx2coa_c + nadh_c --> hxcoa_c + nad_c
Percent,Flux,Reaction,Definition
100.00%,-0.399,ACACT3r,accoa_c + hxcoa_c <=> 3oocoa_c + coa_c


The cofactor of the Acyl-CoA dehydrogenase is a flavoprotein according to literature. The "f" version is kept.

In [139]:
# Remove the reactions involving NAD/NADH
modelCopy.remove_reactions(['ACOAD2','ACOAD3','ACOAD4','ACOAD5','ACOAD6','ACOAD7'])

# Edit in reaction overview
for rxn in ['ACOAD2','ACOAD3','ACOAD4','ACOAD5','ACOAD6','ACOAD7']:
    rxn_origin_df.loc[rxn, 'added'] = 'removed'
    rxn_origin_df.loc[rxn, 'notes'] = 'Caused unbound flux. Removed in 4.1. %sf was kept as correct reaction.'%rxn

The other reactions did not generate energy in the cycles and were therefore not adapted for the moment.

# 10. Save updated model

In [140]:
# Save the model
cobra.io.json.save_json_model(modelCopy, str('../data/models/%s_cur_4.1.json'%EcN_ID), pretty=False)

In [141]:
# Save as a table
rxn_origin_df.to_csv('../tables/rxn_origin.csv')